In [ ]:
import cv2
import numpy as np

# Διαβάζουμε την εικόνα
image = cv2.imread('leaf.jpg')

# Μετατροπή σε κλίμακα του γκρι
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Κατωφλίωση
threshold_value = 220
_, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)

# Εμφάνιση των εικόνων
cv2.imshow('Grayscale Image', gray_image)
cv2.imshow('Binary Image', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
  
  

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def moore_boundary(image):
    height, width = image.shape
    boundary_image = np.zeros((height, width), dtype=int)

    def is_boundary_pixel(x, y):
        if image[x][y] == 0:
            return False
        for i in range(x - 1, x + 2):
            for j in range(y - 1, y + 2):
                if image[i][j] == 0:
                    return True
        return False

    for x in range(1, height - 1):
        for y in range(1, width - 1):
            if is_boundary_pixel(x, y):
                boundary_image[x][y] = 1

    return boundary_image

boundary_image = moore_boundary(binary_image)

plt.imshow(boundary_image, cmap='gray')

In [ ]:
# Υπολογισμός των περιγραφέων Fourier
adjusted_contour_float32 = boundary_image.astype(np.float32)
fourier_descriptors = cv2.dft(adjusted_contour_float32, flags=cv2.DFT_COMPLEX_OUTPUT)
print(fourier_descriptors)

In [ ]:
# Perform inverse Fourier Transform
inverse_dft = cv2.idft(fourier_descriptors)

# Split the result into real and imaginary parts
real_part, imag_part = cv2.split(inverse_dft)

# Compute magnitude and phase
magnitude = cv2.magnitude(real_part, imag_part)
phase = cv2.phase(real_part, imag_part)

# Optionally, you may normalize the magnitude to get it in the same range as the original image
normalized_magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

# Display or use the result as needed


In [ ]:
import numpy as np
import cv2

# Assuming you already have 'fourier_descriptors' defined

# Calculate the magnitude of each Fourier descriptor
magnitudes = np.abs(fourier_descriptors)

# Sort descriptors based on magnitude (importance)
sorted_indices = np.argsort(magnitudes)[::-1]

# Define the percentages you want to use
percentages = [100, 50, 10, 1]

# Loop through the percentages
for percent in percentages:
    # Calculate the number of descriptors to use based on the percentage
    num_descriptors_to_use = int(percent / 100 * len(sorted_indices))

    # Select the most important descriptors
    selected_indices = sorted_indices[:num_descriptors_to_use]
    selected_descriptors = fourier_descriptors[selected_indices]
    print(len(selected_descriptors))

    # # Convert selected descriptors to complex128 dtype if not already
    # if selected_descriptors.dtype != np.complex128:
    #     selected_descriptors = selected_descriptors.astype(np.complex128)

    # Inverse Fourier transform to reconstruct the contour
    # flags = cv2.DFT_COMPLEX_INPUT | cv2.DFT_REAL_OUTPUT | cv2.DFT_SCALE
    reconstructed_contour = cv2.idft(selected_descriptors)

    # Extract the real part of the reconstructed contour
    reconstructed_contour = np.abs(reconstructed_contour)

    # Normalize the reconstructed contour to the range [0, 255]
    reconstructed_contour = cv2.normalize(reconstructed_contour, None, 0, 255, cv2.NORM_MINMAX)

    # Convert the reconstructed contour to 8-bit unsigned integer
    reconstructed_contour = reconstructed_contour.astype(np.uint8)

    # Print the reconstructed contour and the number of non-zero elements
    print(f"Reconstructed Contour {percent}%:")
    print(reconstructed_contour)
    print(f"Number of non-zero elements: {np.count_nonzero(reconstructed_contour)}")

    # Display the reconstructed contour
    cv2.imshow(f'Reconstructed Contour {percent}%', reconstructed_contour)
    cv2.waitKey(0)

# Close any open image windows
cv2.destroyAllWindows()
